# Chapter4: データをグラフにする・可視化する方法
このセクションでは、PythonとPolarsを用いたデータ分析の結果を解釈し、山梨県の課題を発見する方法を学びます。具体的なデータの解釈方法、課題発見のための視点、そしてそれらを踏まえた新たな可能性について考察します。


In [1]:
# 必要なライブラリのインストール
!pip install polars httpx pandas numpy

## データの読み込み・可視化の準備の例
データの可視化を行う前に、まずは必要なデータを読み込むことから始めましょう。このプロセスでは、データを分析しやすい形に整え、それを基にして様々なグラフやチャートを作成します。データの可視化は、数字や文字だけの情報を直感的に理解しやすくするための重要な手段です。

今回は、日本政府が運営する「[e-Gov データポータル](https://data.e-gov.go.jp/info/ja)」というサイトから特定のデータセットを使用します。このサイトは、政府が持っている様々な統計データや調査結果を公開しており、誰でも自由にアクセスして情報を得ることができます。

### 使用するデータセット

具体的には、「[令和４年国家公務員給与等実態調査の結果_II　統計表](https://data.e-gov.go.jp/data/dataset/jinji_20230119_0013)」というデータセットを例にとってみます。このデータセットには、令和4年（2022年）における国家公務員の給与やその他の待遇に関する詳細な統計情報が含まれています。これを利用することで、公務員の給料の平均値や分布、職種や経験年数による違いなどの洞察を得ることが可能です。

また、この方法は他のデータセットにも応用できます。e-Gov データポータルには様々な種類のデータが公開されているので、興味や必要に応じてURLを変更し、異なるデータセットを読み込むことができます。


In [2]:
import httpx
import pandas as pd
import polars as pl
from io import BytesIO
import ssl

# CSVファイルのURL
url = "https://www.jinji.go.jp/kankoku/kokkou/04kokkoulink/20220200.csv"

# カスタムSSLコンテキストの作成
ssl_context = ssl.create_default_context()
ssl_context.set_ciphers('DEFAULT:@SECLEVEL=1')

# httpxクライアントの設定
try:
    with httpx.Client(verify=ssl_context) as client:
        response = client.get(url)
        # ステータスコードが200（成功）であることを確認
        if response.status_code == 200:
            # BytesIOを使用してバイナリデータをpandasが読み込める形式に変換
            data = BytesIO(response.content)
            # pandasでデータを読み込む（エンコーディングは以前推測したSHIFT_JISを使用）
            df = pd.read_csv(data, encoding='SHIFT_JIS', header=1)
            # 最初の2行をカラム名として組み合わせる
            combined_columns = [f'{a}_{b}' for a, b in zip(df.iloc[0], df.iloc[1])]
            # 新たなDataFrameを作成（最初の2行を除外）
            df = pd.DataFrame(df.values[2:], columns=combined_columns)
            # strをInt64に変換
            columns_to_convert = ['最終学歴計_人', '最終学歴が大学卒_人', '最終学歴が短大卒_人', '最終学歴が高校卒_人', '最終学歴が中学卒_人']
            for column in columns_to_convert:
                df[column] = df[column].astype("Int64")
            # polarsに変換
            df = pl.from_pandas(df)
            # 読み込んだデータフレームの最初の数行を表示
            print(df.head(5))
        else:
            print(f"データの取得に失敗しました。ステータスコード: {response.status_code}")
except Exception as e:
    print(f"データの取得中にエラーが発生しました: {e}")


shape: (5, 7)
┌──────────────┬──────────┬──────────────┬──────────────┬──────────────┬─────────────┬─────────────┐
│ nan_俸給表   ┆ nan_性別 ┆ 最終学歴計_  ┆ 最終学歴が大 ┆ 最終学歴が短 ┆ 最終学歴が  ┆ 最終学歴が  │
│ ---          ┆ ---      ┆ 人           ┆ 学卒_人      ┆ 大卒_人      ┆ 高校卒_人   ┆ 中学卒_人   │
│ str          ┆ str      ┆ ---          ┆ ---          ┆ ---          ┆ ---         ┆ ---         │
│              ┆          ┆ i64          ┆ i64          ┆ i64          ┆ i64         ┆ i64         │
╞══════════════╪══════════╪══════════════╪══════════════╪══════════════╪═════════════╪═════════════╡
│ 計           ┆  計      ┆ 253401       ┆ 142626       ┆ 36226        ┆ 74354       ┆ 195         │
│ 計           ┆  男      ┆ 197769       ┆ 110774       ┆ 27373        ┆ 59457       ┆ 165         │
│ 計           ┆  女      ┆ 55632        ┆ 31852        ┆ 8853         ┆ 14897       ┆ 30          │
│ 行政職俸給表 ┆  計      ┆ 139947       ┆ 86294        ┆ 17469        ┆ 36160       ┆ 24          │
│ （一)        ┆          ┆           

### 棒グラフの作成

では実際にデータの可視化を行ってみましょう。このコードサンプルは、`plotly.express`を用いて、データフレーム`df`の特定の行（3行目から6行目）に含まれるデータを基に、最終学歴に基づく人数分布のバーチャートを生成します。

- **グラフの内容**: X軸に性別（`nan_性別`列）、Y軸に人数（複数の最終学歴カテゴリーの人数）、そして`nan_俸給表`列に基づいてサブプロットを分けています。
- **カテゴリー**: 最終学歴計、大学卒、短大卒、高校卒、中学卒の人数をグループ化して表示します。
- **カスタマイズ**: X軸とY軸にタイトルを設定し、凡例のタイトルを「最終学歴」としています。また、ラベルをカスタマイズして、Y軸のラベルを「人数」、バーの変数名を「最終学歴カテゴリー」としています。

In [3]:
import plotly.express as px

fig = px.bar(df[3:6], x='nan_性別', y=['最終学歴計_人', '最終学歴が大学卒_人', '最終学歴が短大卒_人', '最終学歴が高校卒_人', '最終学歴が中学卒_人'],
             title='最終学歴に基づく人数分布',
             labels={'value':'人数', 'variable':'最終学歴カテゴリー'},
             facet_col='nan_俸給表',
             barmode='group')

fig.update_layout(xaxis_title="性別", yaxis_title="人数", legend_title="最終学歴")
fig.show()

## いろいろなデータ可視化手法

### 1. 散布図 (Scatter Plots) - 相関の可視化

散布図は、2つの連続変数の関係性を可視化するのに最適です。相関関係を直感的に把握でき、データポイント間の関係性やトレンドを明確にします。

In [4]:
import plotly.express as px
import numpy as np

# サンプルデータの生成
n = 1000
df = pd.DataFrame({
    '変数A': np.random.randn(n),
    '変数B': np.random.randn(n) + np.arange(n) / 50
})
dl = pl.from_pandas(df)

# 散布図の作成
fig = px.scatter(dl, x='変数A', y='変数B', title='変数Aと変数Bの散布図')
fig.show()


### 2. バブルチャート - 3変数の関係性の可視化

バブルチャートは散布図の拡張で、3つ目の変数の大きさ（重要度や量）をバブルのサイズで表現します。

In [5]:
df['変数C'] = np.random.rand(n) * 100  # サイズ用の変数
dl = pl.from_pandas(df)

fig = px.scatter(dl, x='変数A', y='変数B', size='変数C', color='変数C',
                 title='変数Aと変数Bの関係性に変数Cのサイズを加えたバブルチャート')
fig.show()

### 3. ヒートマップ - 変数間の相関の可視化

ヒートマップは、複数の変数間の相関を色の濃淡で表現し、どの変数が強く関連しているかを一目で把握できます。

In [6]:
# 相関行列の計算
corr = dl.corr()

# ヒートマップの作成
fig = px.imshow(corr, text_auto=True, aspect="auto", title='変数間の相関ヒートマップ')
fig.show()

### 4. ボックスプロット - 分布と外れ値の可視化

ボックスプロットは、データの分布、中央値、四分位数、外れ値を可視化します。複数のグループ間でデータを比較するのに適しています。

In [7]:
df['カテゴリ'] = np.random.choice(['グループA', 'グループB'], n)

fig = px.box(df, x='カテゴリ', y='変数A', title='カテゴリによる変数Aの分布と外れ値の可視化')
fig.show()



### 5. ラインチャート - 時系列データのトレンド分析

時系列データにおける変数のトレンドや季節性を把握するのにラインチャートは有効です。



In [8]:
df['時間'] = pd.date_range(start='1/1/2022', periods=n)

fig = px.line(df, x='時間', y='変数A', title='時間による変数Aのトレンド分析')
fig.show()